# STARS base plots in bqplot

## Loading Data and initial wrangling

In [33]:
# I had to update some jupyter components before installing bqplot
# conda update jupyter_core jupyter_client
# conda install -c conda-forge bqplot
import bqplot as bq
import pandas as pd
import pysal as ps
from libpysal.weights.contiguity import *
import bqplot.pyplot as plt
import geopandas as gpd
import numpy as np
from bqplot.interacts import (
    FastIntervalSelector, IndexSelector, BrushIntervalSelector,
    BrushSelector, MultiSelector, LassoSelector, PanZoom, HandDraw
)
from ipywidgets import ToggleButtons, VBox, HTML
import re
from scipy import stats
from bqplot import * # Imports Figure, Map, Mercator, Orthographic, ColorScale, ColorAxis, AlbersUSA, topo_load, Tooltip, Scatter, Lines, etc.

In [34]:
csv_path = ps.examples.get_path('usjoin.csv')
usjoin = pd.read_csv(csv_path)

In [35]:
years = list(range(1929, 2010))                  
cols_to_calculate = list(map(str, years))

In [36]:
shp_path = ps.examples.get_path('us48.shp')
us48_map = gpd.read_file(shp_path)
us48_map = us48_map[['STATE_FIPS','geometry']]
us48_map.STATE_FIPS = us48_map.STATE_FIPS.astype(int)
df_map = us48_map.merge(usjoin, on='STATE_FIPS')

In [37]:
df_map.head()

,STATE_FIPS,geometry,Name,1929,1930,1931,1932,1933,1934,1935,...,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009
0,53,(POLYGON ((-122.400749206543 48.22539520263672...,Washington,741,658,534,402,376,443,490,...,31528,32053,32206,32934,34984,35738,38477,40782,41588,40619
1,30,POLYGON ((-111.4746322631836 44.70223999023438...,Montana,592,501,382,339,298,364,476,...,22569,24342,24699,25963,27517,28987,30942,32625,33293,32699
2,23,(POLYGON ((-69.77778625488281 44.0740737915039...,Maine,601,576,491,377,371,416,430,...,25623,27068,27731,28727,30201,30721,32340,33620,34906,35268
3,38,POLYGON ((-98.73005676269531 45.93829727172852...,North Dakota,382,311,187,176,146,180,272,...,25068,26118,26770,29109,29676,31644,32856,35882,39009,38672
4,46,POLYGON ((-102.7879333496094 42.99532318115234...,South Dakota,426,366,241,189,129,184,309,...,26115,27531,27727,30072,31765,32726,33320,35998,38188,36499


In [38]:
# Making the dataset tidy
us_tidy = pd.melt(df_map, 
                  id_vars=['Name', 'STATE_FIPS', 'geometry'],
                  value_vars=cols_to_calculate, 
                  var_name='Year', 
                  value_name='Income')

# Function that calculates Per Capita Ratio
def calculate_pcr(x):
    return x / np.mean(x)

# Establishing a contiguity matrix for a specific year. It is the same for all years.
W = Queen.from_dataframe(us_tidy[us_tidy.Year == '1929'])
W.transform = 'r'

# Function that calculates lagged value
def calculate_lag_value(x):
    return ps.lag_spatial(W, x)

# In the first function (calculate_pcr), a series is returned, in the second (calculate_lag_value), an array, so the assign method is used to keep the indexes of the pandas Dataframe

us_tidy['PCR'] = us_tidy.groupby('Year').Income.apply(lambda x: calculate_pcr(x))
us_tidy = us_tidy.assign(Income_Lagged = us_tidy.groupby('Year').Income.transform(calculate_lag_value),
                         PCR_Lagged = us_tidy.groupby('Year').PCR.transform(calculate_lag_value))

In [39]:
us_tidy.head()

,Name,STATE_FIPS,geometry,Year,Income,PCR,Income_Lagged,PCR_Lagged
0,Washington,53,(POLYGON ((-122.400749206543 48.22539520263672...,1929,741,1.204470,587.500000,0.954961
1,Montana,30,POLYGON ((-111.4746322631836 44.70223999023438...,1929,592,0.962276,497.500000,0.808669
2,Maine,23,(POLYGON ((-69.77778625488281 44.0740737915039...,1929,601,0.976905,686.000000,1.115069
3,North Dakota,38,POLYGON ((-98.73005676269531 45.93829727172852...,1929,382,0.620928,539.000000,0.876126
4,South Dakota,46,POLYGON ((-102.7879333496094 42.99532318115234...,1929,426,0.692448,570.833333,0.927870


## Choropleth Map

In [40]:
year = 2009
us_aux = us_tidy[us_tidy.Year == str(year)]
us_aux.head()

,Name,STATE_FIPS,geometry,Year,Income,PCR,Income_Lagged,PCR_Lagged
3840,Washington,53,(POLYGON ((-122.400749206543 48.22539520263672...,2009,40619,1.091761,33098.5,0.889625
3841,Montana,30,POLYGON ((-111.4746322631836 44.70223999023438...,2009,32699,0.878887,37165.5,0.998938
3842,Maine,23,(POLYGON ((-69.77778625488281 44.0740737915039...,2009,35268,0.947937,41882.0,1.125708
3843,North Dakota,38,POLYGON ((-98.73005676269531 45.93829727172852...,2009,38672,1.039430,36706.0,0.986587
3844,South Dakota,46,POLYGON ((-102.7879333496094 42.99532318115234...,2009,36499,0.981024,37972.5,1.020628


In [41]:
base_json = topo_load('map_data/USStatesMap.json')

In [42]:
# List of indexes (ids) of the json file
x = []
for i in range(len(base_json['objects']['subunits']['geometries'])):
    aux = base_json['objects']['subunits']['geometries'][i]['id']
    x.append(aux)

In [43]:
# Some 'states' didn't have value, so I had to create this condition statement
v = []
for i in x:
    if (len(us_aux[us_aux.STATE_FIPS == i].PCR.values.astype(float)) == 1):
        aux = us_aux[us_aux.STATE_FIPS == i].PCR.values.astype(float).item()
    else:
        aux = 0
    v.append(aux)

In [48]:
# There were some Polygons that didn't have 'properties' such as:
# base_json['objects']['subunits']['geometries'][35]['properties']
# This was between Kentucky and Arizona

# This was tricky because I had to make a loop to insert income values of this specific list in only the id's that actually was in the unique list of the id's of the dataset.

for i in range(len(x)):
    if (pd.Series(base_json['objects']['subunits']['geometries'][i]['id']).isin(us_aux.STATE_FIPS).values[0]):
        base_json['objects']['subunits']['geometries'][i]['properties']['PCR_Value'] = v[i]

Converting to a Choropleth map.

In [49]:
sc_geo = AlbersUSA()

# In the example of bqplot the colors of the map_styles relates the 'id' of the json file and a 'variable number' of the color
sc_c1 = ColorScale(scheme='YlOrRd')
axis = ColorAxis(scale=sc_c1)

color_dict = dict(zip(us_aux.STATE_FIPS, us_aux.PCR))

map_styles = {'color': color_dict,
              'scales': {'projection': sc_geo, 'color': sc_c1}, 
              'colors': {'default_color': 'Grey'}}

def_tt = Tooltip(fields=['name', 'PCR_Value'], formats = ['','0.3f'], labels = ['Name: ', 'PCR: '])
choro_map = Map(map_data=base_json, **map_styles, tooltip=def_tt)
choro_map.interactions = {'click': 'select', 'hover': 'tooltip'}
choro_map = Figure(marks=[choro_map], 
                   axes=[axis], 
                   title='Choropleth Example for ' + str(year), 
                   fig_margin={'top': 0, 'bottom': 0, 'left': 0, 'right': 0})
choro_map

Figure(axes=[ColorAxis(scale=ColorScale(scheme='YlOrRd'))], fig_margin={'top': 0, 'bottom': 0, 'left': 0, 'rig…

## Scatter plot

### Basic

In [52]:
scat_Var = us_aux.PCR
scat_VarLag = us_aux.PCR_Lagged

bs_sc_x = bq.LinearScale(min = min(us_tidy.PCR), max = max(us_tidy.PCR))
bs_sc_y = bq.LinearScale(min = min(us_tidy.PCR_Lagged), max = max(us_tidy.PCR_Lagged))
bs_ax_x = bq.Axis(label='Original Variable', scale=bs_sc_x)
bs_ax_y = bq.Axis(label='Lagged Variable', scale=bs_sc_y, orientation='vertical')

index = us_aux.Name.index('California')
bs_sc_colors = LinearScale()
test = ['blue'] * len(scat_Var)
test[index] = 'red'

bs_sc_sizes = LinearScale(min = -1, max = 1)
test2 = [0] * len(scat_Var)
test2[index] = 5

# The 'names' field shoul be 'name' in bqplot: https://github.com/bloomberg/bqplot/blob/master/examples/Applications/Wealth%20of%20Nations.ipynb
scatt_tt = Tooltip(fields = ['name', 'x', 'y'], labels = ['Name: ', 'PCR: ', 'PCR Lagged: '], formats = ['', '0.3f', '0.3f'])

scatt_plot = Scatter(x = scat_Var, 
                     y = scat_VarLag, 
                     names = us_aux.Name.tolist(),
                     labels = us_aux.Name.tolist(),
                     scales = {'x': bs_sc_x, 
                               'y': bs_sc_y,
                               'colors': bs_sc_colors,
                               'size': bs_sc_sizes}, 
                     colors = test,
                     size = test2,
                     tooltip = scatt_tt,
                     unhovered_style={'opacity': 0.25}
                    )

b,a = np.polyfit(scat_Var, scat_VarLag, 1)
x_reg = [min(us_tidy.PCR), max(us_tidy.PCR)]
y_reg = [a + i * b for i in x_reg]

reg_line = Lines(x = x_reg, 
                 y = y_reg, 
                 scales={'x': bs_sc_x, 
                         'y': bs_sc_y}, 
                 colors = ['black'])

basic_scatter = bq.Figure(axes=[bs_ax_x, bs_ax_y], marks=[scatt_plot, reg_line], title = 'Scatterplot for '+ str(year))
basic_scatter

TypeError: 'Int64Index' object is not callable

In [55]:
us_aux.Name.values.index

'Washington'

### With dynamic regression from box select

In [15]:
default_marks = [scatt_plot, reg_line]
br_sel_scat = BrushSelector(x_scale=bs_sc_x, y_scale=bs_sc_y, marks=default_marks, color='red')
db_scat_brush = HTML(value = '[]')

## call back for the selector
def brush_callback_scat(change):
    if (len(br_sel_scat.selected) == 0):
        dynamic_scatter.marks = [scatt_plot, reg_line]
    else:
        db_scat_brush.value = str(br_sel_scat.selected)
        brush_box_coordinates = re.sub('[\[\]]', '', db_scat_brush.value).split(', ')
        x_bottom_left = float(brush_box_coordinates[0])
        y_bottom_left = float(brush_box_coordinates[1])
        x_upper_right = float(brush_box_coordinates[2])
        y_upper_right = float(brush_box_coordinates[3])
        conditions = (scat_Var > x_bottom_left) & (scat_Var < x_upper_right) & (pd.Series(scat_VarLag) < y_upper_right) & (pd.Series(scat_VarLag) > y_bottom_left)
        sub_scat_Var = scat_Var.loc[conditions]
        sub_scat_VarLag = pd.Series(scat_VarLag).loc[conditions]
        sub_b,sub_a = np.polyfit(sub_scat_Var, sub_scat_VarLag, 1)
        sub_x_reg = [min(scat_Var), max(scat_Var)]
        sub_y_reg = [sub_a + i * sub_b for i in x_reg]
        sub_reg_line = Lines(x = sub_x_reg, y = sub_y_reg, scales={'x': bs_sc_x, 'y': bs_sc_y}, colors = ['red'])
        sub_reg_line = sub_reg_line
        dynamic_scatter.marks = [scatt_plot, reg_line, sub_reg_line]
    
br_sel_scat.observe(brush_callback_scat, names=['brushing'])

dynamic_scatter = bq.Figure(axes=[bs_ax_x, bs_ax_y], 
                            marks = default_marks, 
                            title = 'Scatterplot for '+ str(year), 
                            interaction = br_sel_scat)
dynamic_scatter

Figure(axes=[Axis(label='Original Variable', scale=LinearScale(max=1.9341917024320459, min=0.36337625178826893…

TypeError: expected non-empty vector for x

## Time Path plot

In this plot, we have to select a specific state.

In [16]:
state_selected = 'California'
us_tidy_aux = us_tidy[us_tidy.Name == state_selected]

Var = us_tidy_aux.PCR
VarLag = us_tidy_aux.PCR_Lagged

In [17]:
tp_sc_x = LinearScale()
tp_sc_y = LinearScale()

tp_line = Lines(x = Var, y = VarLag, scales={'x': tp_sc_x, 'y': tp_sc_y})
tp_ax_x = Axis(scale = tp_sc_x, label = 'Original Variable', tick_style= {'font-size': '7px'})
tp_ax_y = Axis(scale = tp_sc_y, orientation = 'vertical', label = 'Lagged Variable')

sc_size = LinearScale()
test = [0] * len(Var)
test[0] = 1

tp_scatt = Scatter(x = Var, 
                   y = VarLag, 
                   size = test,
                   scales = {'x': tp_sc_x, 
                             'y': tp_sc_y,
                             'size': sc_size}, 
                   colors = ['black'])

tp_figure = Figure(marks = [tp_line, tp_scatt], axes = [tp_ax_x, tp_ax_y], title = 'Time Path for ' + state_selected)
tp_figure

Figure(axes=[Axis(label='Original Variable', scale=LinearScale(), tick_style={'font-size': '7px'}), Axis(label…

## Time Series of Global Moran's I

First, we need to calculate the Moran's I value for all years.

In [18]:
# Calculating Moran'I for every column
morans = []
for i in cols_to_calculate:
    aux = ps.Moran(df_map[i], W).I
    morans.append(aux)

In [19]:
ts_sc_x = LinearScale()
ts_sc_y = LinearScale()

moran_line = Lines(x = years, y = morans, scales={'x': ts_sc_x, 'y': ts_sc_y})

moran_tt = Tooltip(fields=['x', 'y'], labels = ['Year: ', 'Moran Value: '], formats = ['f', '0.3f'])

moran_scatt = Scatter(x = years, 
                      y = morans, 
                      scales = {'x': ts_sc_x, 'y': ts_sc_y}, 
                      colors = ['black'],
                      tooltip = moran_tt,
                      unhovered_style={'opacity': 0.25})
moran_scatt.default_size = 10

ts_ax_x = Axis(scale = ts_sc_x, label='Years', tick_style= {'font-size': '7px'})
ts_ax_y = Axis(scale = ts_sc_y, orientation='vertical', label='Moran\'s I')

ts_plot = Figure(marks=[moran_line, moran_scatt], axes=[ts_ax_x, ts_ax_y], title='Moran\'s I Time Series plot')
ts_plot

Figure(axes=[Axis(label='Years', scale=LinearScale(), tick_style={'font-size': '7px'}), Axis(label="Moran's I"…

## Density Plot

First, let's estimate the density curve for a grid.

In [20]:
year_base = '1929'
us_aux_dens = us_tidy[us_tidy.Year == str(year_base)]
value_vetor = us_aux_dens.PCR.values
kde1 = stats.gaussian_kde(value_vetor, bw_method = 'silverman')


year2 = max(us_tidy.Year)
value_vetor2 = us_tidy[us_tidy.Year == str(year2)].PCR.values

kde2 = stats.gaussian_kde(value_vetor2, bw_method = 'silverman')

grid = np.linspace(start = min(min(value_vetor), min(value_vetor2)), 
                   stop  = max(max(value_vetor), max(value_vetor2)), 
                   num   = 10000)
dens1 = kde1.evaluate(grid)
dens2 = kde2.evaluate(grid)

In [21]:
dp_sc_x = LinearScale()
dp_sc_y = LinearScale()

density_line1 = Lines(x = grid, y = dens1, scales={'x': dp_sc_x, 'y': dp_sc_y}, fill = 'bottom', fill_opacities = [0.3])
density_line2 = Lines(x = grid, y = dens2, scales={'x': dp_sc_x, 'y': dp_sc_y}, fill = 'bottom', fill_opacities = [0.3], fill_colors = ['red'], colors = ['red'])
dp_ax_x = Axis(scale = dp_sc_x, label = 'Values')
dp_ax_y = Axis(scale = dp_sc_y, orientation = 'vertical', label = 'Density')

density_plot = Figure(marks = [density_line1, density_line2], axes = [dp_ax_x, dp_ax_y], title = 'Density plot for ' + str(year_base) + ' and ' + str(year2))
density_plot.interpolation = 'basis' # In order to make to curve smoother
density_plot

Figure(axes=[Axis(label='Values', scale=LinearScale()), Axis(label='Density', orientation='vertical', scale=Li…

## Boxplot (with scatter markers)

In [22]:
bp_sc_x = LinearScale(min = year - 1, max = year + 1)
bp_sc_y = LinearScale(min = min(us_aux.PCR.values).astype(float), max = max(us_aux.PCR.values).astype(float))
bp_ax_x = Axis(label='Year of ' + str(year), scale = bp_sc_x, tick_style= {'font-size': '5px'})
bp_ax_y = Axis(label='Variable Label', scale = bp_sc_y, orientation='vertical')

x_box = [year]
boxes = Boxplot(x = x_box, y = [us_aux.PCR.values], scales = {'x': bp_sc_x, 'y': bp_sc_y},
                box_fill_color = 'gray', outlier_fill_color = 'black')

In [23]:
# Boxplot with markers
x_scat_box = [year] * len(us_aux.PCR.values)
y_scat_box = us_aux.PCR.values.tolist()
scatt_box = Scatter(x = x_scat_box, y = y_scat_box, scales={'x': bp_sc_x, 'y': bp_sc_y}, colors=['black'])
scatt_box.default_size = 12
boxplot_marks = bq.Figure(axes=[bp_ax_x, bp_ax_y], marks=[boxes, scatt_box], title = 'Boxplot of PCR in '+ str(year))
boxplot_marks

Figure(axes=[Axis(label='Year of 2009', scale=LinearScale(max=2010.0, min=2008.0), tick_style={'font-size': '5…

# Some interactions

## Choropleth with Time Path

# Choropleth event

In [24]:
us_tidy_aux2 = us_tidy[us_tidy.Year == '1960']
dot_index = us_tidy_aux2.Name.tolist().index('Nevada')
dot_index

21

In [25]:
def hover_callback(name, value):
    state_hovered = value['data']['name']
    us_tidy_aux = us_tidy[us_tidy.Name == state_hovered]
    Var = us_tidy_aux.PCR
    VarLag = us_tidy_aux.PCR_Lagged
    tp_line.x = Var
    tp_line.y = VarLag
    tp_scatt.x = Var
    tp_scatt.y = VarLag
    tp_figure.title = 'Time Path for ' + state_hovered
    
    year_active = basic_scatter.title[-4:]
    us_tidy_aux2 = us_tidy[us_tidy.Year == year_active]
    
    Var2 = us_tidy_aux2.PCR
    
    dot_index = us_tidy_aux2.Name.tolist().index(state_hovered)
    
    test = ['blue'] * len(Var)
    test[dot_index] = 'red'
    scatt_plot.colors = test

    test2 = [0] * len(Var)
    test2[dot_index] = 5
    scatt_plot.size = test2

    return state_hovered
    
choro_map_tp = Map(map_data = base_json, **map_styles, tooltip=tp_figure)
choro_map_tp.on_hover(hover_callback)

In [26]:
choro_tp = Figure(marks=[choro_map_tp], 
                  axes=[axis], 
                  title='Choropleth with TimePath', 
                  fig_margin={'top': 0, 'bottom': 0, 'left': 0, 'right': 0})
choro_tp

Figure(axes=[ColorAxis(scale=ColorScale(scheme='YlOrRd'))], fig_margin={'top': 0, 'bottom': 0, 'left': 0, 'rig…

## Auxiliar Map function

In [27]:
def create_map_data_from_year(year):
    us_aux = us_tidy[us_tidy.Year == str(year)]
    base_json = topo_load('map_data/USStatesMap.json')
    x = []
    for i in range(len(base_json['objects']['subunits']['geometries'])):
        aux = base_json['objects']['subunits']['geometries'][i]['id']
        x.append(aux)

    v = []
    for i in x:
        if (len(us_aux[us_aux.STATE_FIPS == i].PCR.values.astype(float)) == 1):
            aux = us_aux[us_aux.STATE_FIPS == i].PCR.values.astype(float).item()
        else:
            aux = 0
        v.append(aux)

    for i in range(len(x)):
        if (pd.Series(base_json['objects']['subunits']['geometries'][i]['id']).isin(us_aux.STATE_FIPS).values[0]):
            base_json['objects']['subunits']['geometries'][i]['properties']['PCR_Value'] = v[i]

    final_json = base_json
    
    return final_json

### Building a dictionary that contains all the json for all years to optimize the choropleth update

In [28]:
all_json = dict()
for i in range(int(min(us_tidy.Year)), int(max(us_tidy.Year)) + 1):
    all_json[i] = create_map_data_from_year(i)

## Time Series Event

In [29]:
def hover_ts(x, value):
    year_hovered = value['data']['x']
    us_aux = us_tidy[us_tidy.Year == str(year_hovered)]
    scat_Var = us_aux.PCR
    scat_VarLag = us_aux.PCR_Lagged   
    b,a = np.polyfit(scat_Var, scat_VarLag, 1)
    y_reg = [a + i * b for i in x_reg]
    scatt_plot.x = scat_Var
    scatt_plot.y = scat_VarLag 
    reg_line.y = y_reg
    basic_scatter.title = 'Scatterplot for ' + str(year_hovered)
    
    scatt_box.x = [year_hovered] * len(scat_Var)
    scatt_box.y = scat_Var
    boxes.x = [year_hovered]
    boxes.y = [scat_Var]
    #boxes.scales['y'] = LinearScale(min = min(scat_Var), max = max(scat_Var))
    #scatt_box.scales['x'] = LinearScale(max=year_hovered + 1, min=year_hovered - 1)
    #scatt_box.scales['y'] = LinearScale(min = min(scat_Var), max = max(scat_Var))
    #boxplot_marks.axes[0] = Axis(label='Year of ' + str(year_hovered), scale = LinearScale(min = year_hovered - 1, max = year_hovered + 1))
    #boxplot_marks.axes[1] = Axis(label='PCR', scale = LinearScale(min = min(scat_Var), max = max(scat_Var)), orientation='vertical')
    boxplot_marks.title = 'Boxplot of PCR in ' + str(year_hovered)
    
    value_vetor2 = us_aux.PCR.values
    kde2 = stats.gaussian_kde(value_vetor2, bw_method = 'silverman')
    grid = np.linspace(start = min(min(value_vetor), min(value_vetor2)), 
                       stop  = max(max(value_vetor), max(value_vetor2)), 
                       num   = 10000)
    dens1 = kde1.evaluate(grid)
    dens2 = kde2.evaluate(grid)
    
    density_line1.x = grid
    density_line1.y = dens1
    density_line2.x = grid
    density_line2.y = dens2
    density_plot.title = 'Density plot for ' + str(year_hovered) + ' and ' + str(year_base)
    
    aux = [0] * len(us_tidy.Year.unique())
    aux[year_hovered - int(min(us_tidy.Year))] = 1
    tp_scatt.size = aux
    
    #choro_map_layout.map_data = create_map_data_from_year(year_hovered)
    choro_map_layout.map_data = all_json[year_hovered]
    choro_map_layout.color = dict(zip(us_aux.STATE_FIPS, us_aux.PCR))
    choro_tp.title = 'Choropleth for ' + str(year_hovered)

moran_scatt.default_size = 30
moran_scatt.on_element_click(hover_ts)

# Building layout

In [30]:
choro_map_layout = Map(map_data = base_json, **map_styles, tooltip=def_tt) # Changed tooltip=tp_figure
choro_map_layout.on_element_click(hover_callback)
choro_tp = Figure(marks=[choro_map_layout], 
                  axes=[axis], 
                  title='Choropleth for ' + str(year), 
                  fig_margin={'top': 0, 'bottom': 0, 'left': 0, 'right': 0})

figures = [choro_tp, basic_scatter, tp_figure, ts_plot, density_plot, boxplot_marks]
choro_tp.layout.height = '300px'
for i in figures:
    i.layout.height = '300px'

In [31]:
import numpy as np
from bqplot import *
from IPython.display import Javascript, display, clear_output
import ipywidgets as widgets
display(widgets.VBox([widgets.HBox(figures[0:3]), widgets.HBox(figures[3:6])], align_content = 'stretch'))

In [32]:
scatt_plot.display_names = False